In [ ]:
from deepdiagnostics import models
from deepdiagnostics import data as data_modules
from deepdiagnostics.utils.config import Config
from deepdiagnostics.utils.simulator_utils import register_simulator

from deepdiagnostics.plots import CDFRanks, CoverageFraction, Ranks, TARP

import yaml

In [ ]:
import warnings
warnings.filterwarnings('ignore')

## Introduction to DeepDiagnostics 

DeepDiagnostics is a command-line utility for running simulation-based inference (sbi) methods. 
It works primarily by interacting with a yaml file to overwrite defaults and specify what diagnostics to run and how.

### Configuration 

The configuration files controls most of DeepDiagnostics. 
It is broken into 7 parts. 
* Common

This controls things like the paths of results, where the simulations are registered, and the random seed. 

* Model 

Specify the path and backend for running model inference. 

* Data 

Specify the prior, simulation, data file, and way to read the data file. 

* Plots Common 

Default parameters all plots use unless otherwise specified. 

* Metrics Common 

Default parameters all metrics use unless otherwise specified.

* Plots 

Dictionary of all the plots to generate. Each field of the dictionary is the name of plot and their corresponding `kwargs`. 

* Metrics 

Same concept as plots! 

### Defaults 
The defaults for these fields are as follows: 

In [ ]:
from deepdiagnostics.utils.defaults import Defaults
Defaults

## Running DeepDiagnostics  

Operation has two main modes: Either add command line arguments for key fields or specify a whole new configuration file. 

In [ ]:
! diagnose -h

## Running a simple example

Minimal examples can be run by supplying a model path (and a matching model engine to load it, if using a model format that is not SBI), and data. 
Below is an example for both CLI and standalone execution.
When a simulator is not included, a warning will be shown but this does not impact non-generative metrics and plots.

In [ ]:
! diagnose --model_path ../resources/savedmodels/sbi/sbi_linear_from_data.pkl --data_path ../resources/saveddata/data_validation.h5  --plots CoverageFraction

## Using a simulator

In order to run some of the metrics and plots (including `LC2ST`, `PPC` and `PriorPC`), you must supply a simulator. 
If you do not supply a simulator and try to run these options in CLI a warning will be raised and the other metrics/plots will run as expected. 
In standalone mode, when trying to initialize metrics and plots that require a simulator, without a simulator, `LookupTableSimulator` will be used instead, which may cause inaccuracies. 

Simulators are all subclasses of `data.Simulator`, and need to be registered with `register_simulator` to use during runtime. 
`data.Simulator` is an abstract class that requires a `generate_context` 
(which takes a number of samples and returns a random sample of context the simulator uses to produce results. 
This can either be loaded in from a specific file, or a random distribution.) 
and `simulate` method 
(which takes a context and parameters of the model )
See below for an example with typing, simulating a 2d case where the model being fit is a linear model. 

Please note that the simulator file **must** be saved a python file, so it can be imported and used by metrics and plots. If you write you simulator in a notebook, it cannot be registered. Using the line magic `%%writefle your-sim-here.py`, you can save it from your notebook. 

In [ ]:
%%writefile my_simulator.py 

from deepdiagnostics.utils.simulator_utils import register_simulator
from deepdiagnostics.data.simulator import Simulator
import numpy as np 

class MySimulator(Simulator): 
    def generate_context(self, n_samples: int=101) -> np.ndarray:
        return np.linspace(0, 100, n_samples)
    
    def simulate(self, theta: np.ndarray, context_samples: np.ndarray) -> np.ndarray:
        thetas = np.atleast_2d(theta)
        if thetas.shape[1] != 2:
            raise ValueError("Input tensor must have shape (n, 2) where n is the number of parameter sets.")

        if thetas.shape[0] == 1:
            # If there's only one set of parameters, extract them directly
            m, b = thetas[0, 0], thetas[0, 1]
        else:
            # If there are multiple sets of parameters, extract them for each row
            m, b = thetas[:, 0], thetas[:, 1]
        rs = np.random.RandomState()
        sigma = 1
        epsilon = rs.normal(loc=0, scale=sigma, size=(len(context_samples), thetas.shape[0]))
        
        # Initialize an empty array to store the results for each set of parameters
        y = np.zeros((len(context_samples), thetas.shape[0]))
        for i in range(thetas.shape[0]):
            m, b = thetas[i, 0], thetas[i, 1]
            y[:, i] = m * context_samples + b + epsilon[:, i]
        return y.T

In [ ]:
# Just running the file to make sure we're not either missing imports or have a syntax error
! python3 my_simulator.py

In [ ]:
from my_simulator import MySimulator

register_simulator("MySimulator", MySimulator) 
# We are registering without having a config set ahead of time, so it may raise a warning. This is fine!
# Only reason we'd want to use a config ahead of time is if we were running this in a cluster 
# And had specific requirements where we can put files 
# In which case we'd change the "common":{"sim_location": <path>} field

In [ ]:

my_config = {
    "model": {"model_path": "../resources/savedmodels/sbi/sbi_linear_from_data.pkl"}, 
    "data": {
        "data_path": "../resources/saveddata/data_test.h5", 
        "simulator": "MySimulator"}, 
    "metrics_common": {
        "use_progress_bar": True,
        "samples_per_inference": 100,
        "percentiles": [75, 85, 95],
        "number_simulations": 10}, 
    "metrics": {},
    "plots":{}
}
with open("./my_config.yaml", "w") as f: 
    yaml.safe_dump(my_config, f)

In [ ]:
# Because nothing is set in the metrics or plots in the above config, nothing will run. 
! diagnose --config ./my_config.yaml

In [ ]:
# We can do a similar thing by passing specific kwargs 
# Here we're just calculating the coverage fraction 
! diagnose --model_path ../resources/savedmodels/sbi/sbi_linear_from_data.pkl --data_path ../resources/saveddata/data_test.h5 --simulator MySimulator  --plots CDFParityPlot

This produces a image of the coverage fraction from our model and data, shown below. 

In [ ]:
import os
import matplotlib.pyplot as plt

for plot in os.listdir("./DeepDiagnosticsResources/results/"):
    if not plot.endswith('.png'):
        continue
    print(f"Showing saved plot: {plot}")
    plot_path = "./DeepDiagnosticsResources/results/" + plot
    plt.imshow(plt.imread(plot_path))
    plt.axis('off')
    plt.show()

We can do a similar thing with metrics, or with plot metrics or plots. 

In [ ]:
! diagnose --model_path ../resources/savedmodels/sbi/sbi_linear_from_data.pkl --data_path ../resources/saveddata/data_test.h5 --simulator MySimulator  --metrics CoverageFraction

## Using standalone functions 

DeepDiagnostics, if you have a configuration file set, can also be used with just the functions. Below is a list of all the functions and examples of their use. 

In [ ]:
# All metrics require a model and data 
Config("./my_config.yaml")

model = models.SBIModel("../resources/savedmodels/sbi/sbi_linear_from_data.pkl")
data = data_modules.H5Data("../resources/saveddata/data_test.h5", simulator="MySimulator")

In [ ]:
plot = CDFRanks(model, data, save=False, show=True, run_id="my_run_42")()

In [ ]:
plot = CoverageFraction(model, data, show=True, save=False, run_id="my_run_42")()

In [ ]:
Ranks(model, data, show=True, save=False, run_id="my_run_42")()

In [ ]:
TARP(model, data, save=False, show=True, run_id="my_run_42")(
    coverage_sigma=5, bootstrap_calculation=True
)

## Rerunning Plots

If you want to return plots you have already made and saved, instead of redoing the entire calculation, you can load an intermediate file with all the calculations needed for the plot. 
The file is saved with a run ID in your "results" folder, so if you set a file called `{run_id}_diagnostic_metrics.h5`. 
Supplying this path to the `plot` method of an plot will reload that data and plot it again. 

In [ ]:
from deepdiagnostics.plots import Parity

run_id = "my_run_42"
out_path = "./DeepDiagnosticsResources/results/"

Parity(
	model, data, run_id, save=True, show=False,
	out_dir=out_path,
	parameter_names=["$\theta_1$", "$\theta_2$"], 
	parameter_colors=["#5ec962", "#fde725"]
)(include_residual=True)
calculation_path = os.path.join(
	out_path, f"{run_id}_diagnostic_metrics.h5")

# Change the parameter names and colors, don't rerun metrics
# Return the figure and axes for further customization
figure, axes = Parity(
	model=None, data=None, run_id=run_id, save=False, show=False,
	out_dir=out_path,
	parameter_names=["Parameter 1", "Parameter 2"],
	parameter_colors=["#1f77b4", "#ff7f0e"]
).plot(
	data_display=calculation_path,
	include_residual=True)



## Replicating with command line interface

To do the same thing with the CLI, just supply a config file with the metrics listed

In [ ]:
my_config = {
    "model": {"model_path": "../resources/savedmodels/sbi/sbi_linear_from_data.pkl"}, 
    "data": {
        "data_path": "../resources/saveddata/data_validation.h5", 
        "simulator": "MySimulator"}, 
    "metrics_common": {
        "use_progress_bar": True,
        "samples_per_inference": 1000,
        "percentiles": [75, 85, 95],
        "number_simulations": 50}, 
    "metrics": {},
    "plots":{
        "LC2ST":{}, 
        "TARP":{"coverage_sigma":5, "bootstrap_calculation":True}, 
        "CoverageFraction":{}, 
    }
}
with open("./my_full_config.yaml", "w") as f: 
    yaml.safe_dump(my_config, f)

In [ ]:
! diagnose --config ./my_full_config.yaml

In [ ]:
import os 
os.listdir("./DeepDiagnosticsResources/results")